In [1]:
### importação das bibliotecas ###
import pandas as pd
import numpy as np
from datetime import datetime, date
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp

### Overview do dataset ###
https://www.kaggle.com/datasets/abrambeyer/openintro-possum

O quadro de dados do gambá consiste em nove medições morfométricas em cada um dos 104 gambás da montanha, presos em sete locais do sul de Victoria ao centro de Queensland.

- case --> numero da observação
- site --> o número do local onde o gambá foi preso
- Pop --> população de gambás, seja Vic (Victoria) ou outra (Nova Gales do Sul ou Queensland).
- sex --> gênero: m (macho) ou f (fêmea)
- Age --> idade
- hdlngth --> comprimento da cabeça, em mm
- skullw --> largura do crânio, em mm
- totlngth --> comprimento total, em cm
- taill --> comprimento do pé
- earconch --> comprimento da concha da orelha
- eye --> distância do canto medial para canto lateral de olho direito
- chest --> cintura do peito (em cm)
- belly --> cintura da barriga (em cm)

In [2]:
### carga da base dados ###
df = pd.read_csv('possum.csv')

In [3]:
print('Data Points:',df.shape[0])
print('Atributos',df.shape[1])

Data Points: 104
Atributos 14


In [4]:
df.dtypes

case          int64
site          int64
Pop          object
sex          object
age         float64
hdlngth     float64
skullw      float64
totlngth    float64
taill       float64
footlgth    float64
earconch    float64
eye         float64
chest       float64
belly       float64
dtype: object

In [5]:
df.describe()

,case,site,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
count,104.000000,104.000000,102.000000,104.000000,104.000000,104.000000,104.000000,103.000000,104.000000,104.000000,104.000000,104.000000
mean,52.500000,3.625000,3.833333,92.602885,56.883654,87.088462,37.009615,68.459223,48.130769,15.046154,27.000000,32.586538
std,30.166206,2.349086,1.909244,3.573349,3.113426,4.310549,1.959518,4.395306,4.109380,1.050374,2.045597,2.761949
min,1.000000,1.000000,1.000000,82.500000,50.000000,75.000000,32.000000,60.300000,40.300000,12.800000,22.000000,25.000000
25%,26.750000,1.000000,2.250000,90.675000,54.975000,84.000000,35.875000,64.600000,44.800000,14.400000,25.500000,31.000000
50%,52.500000,3.000000,3.000000,92.800000,56.350000,88.000000,37.000000,68.000000,46.800000,14.900000,27.000000,32.500000
75%,78.250000,6.000000,5.000000,94.725000,58.100000,90.000000,38.000000,72.500000,52.000000,15.725000,28.000000,34.125000
max,104.000000,7.000000,9.000000,103.100000,68.600000,96.500000,43.000000,77.900000,56.200000,17.800000,32.000000,40.000000


In [6]:
df.head(3)

,case,site,Pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,1,1,Vic,m,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,2,1,Vic,f,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,3,1,Vic,f,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0


In [7]:
###  ###
df_vic = df.loc[df['Pop']=='Vic']
df_oth = df.loc[df['Pop']=='other']
df_m = df.loc[df['sex']=='m']
df_f = df.loc[df['sex']=='f']